### This part processed the sensor data: 
- transfered CAL_DATE to Date(as dt)
- add DOW, 
- add total spots & occupied/unoccupied/unknown ratios for different categories.
- processed data is saved seperately so that we can lod in later when needed.

In [2]:
import numpy as np
import pandas as pd
import datetime as dt

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
def display_all(df):
    with pd.option_context("display.max_rows", 1000): 
        with pd.option_context("display.max_columns", 1000): 
            display(df)

In [7]:
sensor_dt = pd.read_csv('sensordt.csv', parse_dates= [12], infer_datetime_format=True)
col_dt = sensor_dt['Date'].copy()
del sensor_dt

In [8]:
sensor = pd.read_csv('ParkingSensorData.csv', parse_dates= True, infer_datetime_format=True)
sensor.drop(sensor.tail(1).index, inplace=True)

In [9]:
sensor['Date'] = col_dt
sensor['DOW'] = sensor['Date'].dt.weekday

In [14]:
def cal_spot(column, df):
    df[column + "_spots"] = df[column]/3600
    
cols = ['OP_TIME', 'OP_TIME', 'NONOP_TIME', 'GMP_TIME', 'COMM_TIME']
for col in cols:
    cal_spot(col, sensor)

In [15]:
def cal_pct(cols, total, df):
    for col in cols:
        df[col+"_pct"] = df[col]/df[total]

pct_cols = [(['TOTAL_OCCUPIED_TIME', 'TOTAL_VACANT_TIME', 'TOTAL_UNKNOWN_TIME'],'TOTAL_TIME'),
            (['OP_OCCUPIED_TIME', 'OP_VACANT_TIME', 'OP_UNKNOWN_TIME'],'OP_TIME'),
            (['NONOP_OCCUPIED_TIME', 'NONOP_VACANT_TIME','NONOP_UNKNOWN_TIME'],'NONOP_TIME'),
            (['GMP_OCCUPIED_TIME','GMP_VACANT_TIME', 'GMP_UNKNOWN_TIME'],'GMP_TIME'),
            (['COMM_OCCUPIED_TIME', 'COMM_VACANT_TIME', 'COMM_UNKNOWN_TIME'],'COMM_TIME')]

for pair in pct_cols:
    cal_pct(pair[0],pair[1],sensor)

In [16]:
sensor.head()

,BLOCK_ID,STREET_NAME,BLOCK_NUM,STREET_BLOCK,AREA_TYPE,PM_DISTRICT_NAME,RATE,START_TIME_DT,TOTAL_TIME,TOTAL_OCCUPIED_TIME,...,OP_UNKNOWN_TIME_pct,NONOP_OCCUPIED_TIME_pct,NONOP_VACANT_TIME_pct,NONOP_UNKNOWN_TIME_pct,GMP_OCCUPIED_TIME_pct,GMP_VACANT_TIME_pct,GMP_UNKNOWN_TIME_pct,COMM_OCCUPIED_TIME_pct,COMM_VACANT_TIME_pct,COMM_UNKNOWN_TIME_pct
0,41522,FILLMORE ST,22.0,FILLMORE ST 2200,Pilot,Fillmore,NaN,01-jul-2012 21:00:00,61200.0,35847.0,...,NaN,0.585735,0.414265,0.0,NaN,NaN,NaN,NaN,NaN,NaN
1,41524,FILLMORE ST,24.0,FILLMORE ST 2400,Pilot,Fillmore,NaN,01-jul-2012 22:00:00,64800.0,27576.0,...,NaN,0.425556,0.574444,0.0,NaN,NaN,NaN,NaN,NaN,NaN
2,50001,JACKSON ST,1.0,JACKSON ST 100,Pilot,Downtown,NaN,01-jul-2012 02:00:00,79200.0,39985.0,...,NaN,0.504861,0.495139,0.0,NaN,NaN,NaN,NaN,NaN,NaN
3,50002,JACKSON ST,2.0,JACKSON ST 200,Pilot,Downtown,NaN,01-jul-2012 08:00:00,82800.0,30465.0,...,NaN,0.367935,0.632065,0.0,NaN,NaN,NaN,NaN,NaN,NaN
4,10903,09TH AVE,3.0,09TH AVE 300,Control,Inner Richmond,NaN,01-jul-2012 08:00:00,93600.0,85690.0,...,NaN,0.915491,0.084509,0.0,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
sensor = sensor.drop(['TOTAL_TIME','TOTAL_OCCUPIED_TIME', 'TOTAL_VACANT_TIME', 'TOTAL_UNKNOWN_TIME',
       'OP_TIME', 'OP_OCCUPIED_TIME', 'OP_VACANT_TIME', 'OP_UNKNOWN_TIME',
       'NONOP_TIME', 'NONOP_OCCUPIED_TIME', 'NONOP_VACANT_TIME',
       'NONOP_UNKNOWN_TIME', 'GMP_TIME', 'GMP_OCCUPIED_TIME',
       'GMP_VACANT_TIME', 'GMP_UNKNOWN_TIME', 'COMM_TIME',
       'COMM_OCCUPIED_TIME', 'COMM_VACANT_TIME', 'COMM_UNKNOWN_TIME'], axis = 1)

In [20]:
sensor.to_csv("agg_sensor.csv")